## Run Constant Erosion Simulations
<br>
  
#### Syntax
`CalcPars, RunPars, CosmoPars, HeliumPars = RunSimulationsConstant(InputPars)` <br>
#### Input 
`InputPars` : dictionary containing the following input parameters: <br>
    - Total Time : total time to run the model || (yrs) || int <br>
    - Erosion Rate : the given (input) erosion rate; constant erosion will be exactly this value || (cm yr-1) || float <br>
    - c : Pareto distribution shape value; should be 1 > x > 0 || || float <br>
    - MAT : mean annual temperature || (deg C) || float <br>
    - T_AMP : seasonal temperature amplitude || (deg C) || float <br>
#### Output
`CalcPars` : dictionary of calculated concentrations and values for "samples" <br>
`RunPars` : dictionary of parameters relevant for the entire model run <br>
`CosmoPars` : dictionary of parameters relevant for CRN calculatiosn <br>
`HeliumPars` : dictionary containing parameters relevant for He3 calcualtions
#### Notes
**Date of Creation:** 7. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Updates:** <br>
30.07.21 : Added units and variable types to function description.<br>
23.09.21 : Changed calc pars from crnLoopFunction to crnLoopFunctionmtest in order to get complete
muon production. <br>

In [1]:
def RunSimulationsConstant(InputPars):
    CalcPars = {}
    MeasPars = {}
    TruePars = {}

    RunPars, CosmoPars, HeliumPars = SetParams(InputPars)
    RunPars = ConstantERGenerator(RunPars)
    RunPars['Scenarios'] = 1
    RunPars['TrueErosionRates'] = RunPars['erosion_rate']
   
    CosmoPars['CRNz'] = np.arange(0,RunPars['max_depths'][1],dz)
    CosmoPars['CRNnz'] = int(RunPars['max_depths'][1] / dz)

    HeliumPars['Hez'] = np.arange(0,RunPars['max_depths'][0],dz)
    HeliumPars['nz'] = int((RunPars['max_depths'][0]) / dz)
    HeliumPars['T0'] = np.array([InputPars['MAT']] * HeliumPars['thermnz'])
    HeliumPars['thermdz'] = (RunPars['max_depths'][0]) / (HeliumPars['thermnz'] - 1)
    
    CalcPars = crnLoopFunctionmtest(RunPars, CosmoPars, CalcPars)
    CalcPars = HeLoopFxn(RunPars, CosmoPars, HeliumPars, CalcPars)
    CalcPars = CNSamplingFxn(RunPars,CosmoPars,HeliumPars,CalcPars)
    
    return CalcPars, RunPars, CosmoPars, HeliumPars